In [195]:
import os
import struct

import cv2
import numpy as np
from docopt import docopt

import cine
from linLUT import linLUT

import glob, os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage import feature
from skimage.filters import threshold_otsu

from SelectFrameMat import select_frame

from CineRead import frame_reader
from CineRead import create_raw_array
from CineRead import unpack_10bit
from CineRead import read_header


def generate_images(file):
    SelectedFrame = select_frame(file, object_threshold = None)
    #print SelectedFrame
    header = read_header(file)

    Out_image = frame_reader(file,header, start_frame=SelectedFrame, count = 1)
    if SelectedFrame > 1:
        Bg_image = frame_reader(file,header, start_frame=1, count = 1)
    else:
        Bg_image = frame_reader(file,header, start_frame=2, count = 1)
    #plt.imshow(Out_image)
    #plt.imsave("/Users/mahdokhtmasaeli/Documents/Deepcell/Data/class-rbc-images/"+str(i), Out_image, cmap=plt.cm.gray)

            
    return Out_image, Bg_image
        
        
def background_subtraction(Out_image, Bg_image):
    threshBg = threshold_otsu(Bg_image)
    Bgbinary = Bg_image > threshBg

    threshOut = threshold_otsu(Out_image)
    Outbinary = Out_image > threshOut

    Bg = np.bitwise_or(Outbinary,Bgbinary)
    BgSubtractedImg = np.bitwise_xor(Outbinary,Bg)   
    return BgSubtractedImg

def find_xy_limits(BgSubtractedImg):
    imarrayDiff_x = np.sum(BgSubtractedImg,0)
    indobject_x = np.where(is_outlier(imarrayDiff_x)==True)
    #xlimits = [np.array(indobject_x)[0,0],np.array(indobject_x)[0,-1]]
    xlim1 = (np.array(indobject_x)[0,0]+np.array(indobject_x)[0,-1])/2
    xlimits = [xlim1-30, xlim1+30]
    
    imarrayDiff_y = np.sum(BgSubtractedImg,1)
    indobject_y = np.where(is_outlier(imarrayDiff_y)==True)

    derivchange = np.diff(indobject_y)
    sortedind = sorted(range(len(derivchange[0,:])), key=derivchange[0,:].__getitem__) 
    ind1 = sortedind[-1]
    ind2 = sortedind[-2]
    ylim1 = np.array(indobject_y)[0,min(ind1,ind2)+1]
    ylim2 = np.array(indobject_y)[0,max(ind1,ind2)]
    #ylimits = [ylim1,ylim2]
    ylim1 = (np.array(indobject_y)[0,min(ind1,ind2)+1]+np.array(indobject_y)[0,max(ind1,ind2)])/2
    ylimits = [ylim1-30, ylim1+30]
    
    return xlimits, ylimits


def crop_img(Img, xlimits, ylimits, borders=5):
    
    cropped_img = Img[ylimits[0]-borders:ylimits[1]+borders,xlimits[0]-borders:xlimits[1]+borders]
    
    return cropped_img

In [168]:
dir = "/Users/mahdokhtmasaeli/Documents/DeepcellVideo/"
Out_image, Bg_image = generate_images(file)
imgdiff = background_subtraction(Out_image,Bg_image)
xlimits, ylimits = find_xy_limits(imgdiff)
cropped_img = crop_img(Out_image,xlimits,ylimits)

In [ ]:
dir = "/Volumes/My Book/Deepcell/class-10umbead/"
os.chdir(dir)
i = 0
for file in glob.glob("*.cine"):
    i += 1
    try:
        Out_image, Bg_image = generate_images(file)
        imgdiff = background_subtraction(Out_image,Bg_image)
        xlimits, ylimits = find_xy_limits(imgdiff)
        cropped_img = crop_img(Out_image,xlimits,ylimits)
        plt.imshow(cropped_img)
        plt.imsave("/Users/mahdokhtmasaeli/Documents/Deepcell/Data/class-10umbead-images-cropped/"+str(i), cropped_img, cmap=plt.cm.gray)

    except:
        print("Unexpected error")  
                        
            

        

Unexpected error
Unexpected error
Unexpected error
Unexpected error
Unexpected error
Unexpected error
Unexpected error
Unexpected error
Unexpected error
Unexpected error
Unexpected error
Unexpected error
Unexpected error
Unexpected error
Unexpected error
Unexpected error
Unexpected error

[446, 546]

[-16, 84]

(array([ 22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  34,  40,  43,
         68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,
         81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
         94,  95,  96,  97,  98,  99, 156, 157, 158, 159, 160]),)

2

68